In [1]:
import time
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.model_selection import StratifiedKFold

def gen_rand(n_size=1):
    '''
    This function return a n_size-dimensional random vector.
    '''
    return np.random.random(n_size)

class NN_DE(object):
    
    def __init__(self, n_pop=10, n_neurons=5, F=0.4, Cr=0.9, p=1, change_scheme=True ,scheme='rand',
                 bounds=[-1, 1], max_sp_evals=np.int(1e5), sp_tol=1e-2):
        #self.n_gens=n_gens
        self.n_pop=n_pop
        self.n_neurons=n_neurons
        self.F=F*np.ones(self.n_pop)
        self.Cr=Cr*np.ones(self.n_pop)
        self.bounds=bounds
        self.p=p
        self.scheme=scheme
        self.change_schame=change_scheme
        self.max_sp_evals=max_sp_evals
        self.sp_tol = sp_tol
        self.sp_evals=0
        self.interactions=0
        # Build generic model
        model = Sequential()
        model.add(Dense(self.n_neurons, input_dim=100, activation='tanh'))
        model.add(Dense(1, activation='tanh'))
        model.compile( loss='mean_squared_error', optimizer = 'rmsprop', metrics = ['accuracy'] )
        self.model=model
        self.change_schame=False
        self.n_dim=model.count_params()
        #self.population=NN_DE.init_population(self, pop_size=self.n_pop,
        #                                             dim=self.n_dim, bounds=self.bounds)
        #self.train_dataset= train_dataset
        #self.test_dataset= test_dataset
        
    def init_population(self, pop_size, dim, bounds=[-1,1]):
        '''
        This function initialize the population to be use in DE
        Arguments:
        pop_size - Number of individuals (there is no default value to this yet.).
        dim - dimension of the search space (default is 1).
        bounds - The inferior and superior limits respectively (default is [-1, 1]).
        '''
        return np.random.uniform(low=bounds[0], high=bounds[1], size=(pop_size, dim))

    def keep_bounds(self, pop, bounds, idx):
        '''
        This function keep the population in the seach space
        Arguments:
        pop - Population;
        bounds - The inferior and superior limits respectively
        '''
        #up_ = np.where(pop>bounds[1])
        #down_ = np.where(pop<bounds[1])
        #best_ = pop[idx]
        #print(pop[pop<bounds[0]])
        #print(down_)
        #print(best_.shape)
        pop[pop<bounds[0]] = bounds[0]; pop[pop>bounds[1]] = bounds[1]
        #pop[pop<bounds[0]] = 0.5*(bounds[0]+best_[down_]); pop[pop>bounds[1]] = 0.5*(bounds[1]+best_[up_])
        return pop

    # Define the Fitness to be used in DE
    def sp_fitness(self, target, score):
        '''
        Calculate the SP index and return the index of the best SP found

        Arguments:
        target: True labels
        score: the predicted labels
        '''
        from sklearn.metrics import roc_curve

        fpr, tpr, thresholds = roc_curve(target, score)
        jpr = 1. - fpr
        sp = np.sqrt( (tpr  + jpr)*.5 * np.sqrt(jpr*tpr) )
        idx = np.argmax(sp)
        return sp[idx], tpr[idx], fpr[idx]#sp, idx, sp[idx], tpr[idx], fpr[idx]

    def convert_vector_weights(self, pop, nn_model):
        
        model = nn_model
        
        generic_weights = model.get_weights()
        hl_lim = generic_weights[0].shape[0]*generic_weights[0].shape[1]
        
        w = []
        hl = pop[:hl_lim]
        ol = pop[hl_lim+generic_weights[1].shape[0]:hl_lim+generic_weights[1].shape[0]+generic_weights[1].shape[0]] 
        w.append(hl.reshape(generic_weights[0].shape))
        w.append(pop[hl_lim:hl_lim+generic_weights[1].shape[0]])
        w.append(ol.reshape(generic_weights[2].shape))
        w.append(np.array(pop[-1]).reshape(generic_weights[-1].shape))
        
        return w
        
    def set_weights_to_keras_model_and_compute_fitness(self,pop, data, nn_model):
        '''
        This function will create a generic model and set the weights to this model and compute the fitness.
        Arguments:
        pop - The population of weights.
        data - The samples to be used to test.
        '''
        fitness = np.zeros((pop.shape[0],3))
        #test_fitness = np.zeros((pop.shape[0],3))
        model=nn_model
        
        if pop.shape[0]!= self.n_pop:
            #print('Local seach ind...')
            w = NN_DE.convert_vector_weights(self, pop=pop, nn_model=model)
            model.set_weights(w)
            y_score = model.predict(data[0])
            fitness = NN_DE.sp_fitness(self, target=data[1], score=y_score)
            
            # Compute the SP for test in the same calling to minimeze the evals
            #test_y_score = model.predict(test_data[0])
            #test_fitness = NN_DE.sp_fitness(self, target=test_data[1], score=test_y_score)
            return fitness#, test_fitness
        
        for ind in range(pop.shape[0]):
            w = NN_DE.convert_vector_weights(self, pop=pop[ind], nn_model=model)
            model.set_weights(w)
            y_score = model.predict(data[0])
            fitness[ind] = NN_DE.sp_fitness(self, target=data[1], score=y_score)
            
            # Compute the SP for test in the same calling to minimeze the evals
            #test_y_score = model.predict(test_data[0])
            #test_fitness[ind] = NN_DE.sp_fitness(self, target=test_data[1], score=test_y_score)
            #print('Population ind: {} - SP: {} - PD: {} - PF: {}'.format(ind, fitness[ind][0], fitness[ind][1], fitness[ind][2]))
        return fitness#, test_fitness


    def evolution(self, dataset):
        
        self.population=NN_DE.init_population(self, pop_size=self.n_pop,
                                              dim=self.n_dim, bounds=self.bounds)
        r_NNDE = {}
        fitness = NN_DE.set_weights_to_keras_model_and_compute_fitness(self, pop=self.population,
                                                                       data=dataset,
                                                                       nn_model=self.model)
        best_idx = np.argmax(fitness[:,0])
        
        # Create the vectors F and Cr to be adapted during the interactions
        NF = np.zeros_like(self.F)
        NCr = np.zeros_like(self.Cr)
        # Create a log
        r_NNDE['log'] = []
        r_NNDE['log'].append((self.sp_evals, fitness[best_idx], np.mean(fitness, axis=0),
                             np.std(fitness, axis=0), np.min(fitness, axis=0), np.median(fitness, axis=0), self.F, self.Cr))

        #r_NNDE['test_log'] = []
        #r_NNDE['test_log'].append((self.sp_evals, test_fitness[best_idx], np.mean(test_fitness, axis=0),
                             #np.std(test_fitness, axis=0), np.min(test_fitness, axis=0), np.median(test_fitness, axis=0), self.F, self.Cr))

        while self.sp_evals < self.max_sp_evals:
            # ============ Mutation Step ===============
            mutant = np.zeros_like(self.population)
            for ind in range(self.population.shape[0]):
                if gen_rand() < 0.1:
                    NF[ind] = 0.2 +0.2*gen_rand()
                else:
                    NF[ind] = self.F[ind]
                tmp_pop = np.delete(self.population, ind, axis=0)
                choices = np.random.choice(tmp_pop.shape[0], 1+2*self.p, replace=False)
                diffs = 0
                for idiff in range(1, len(choices), 2):
                    diffs += NF[ind]*((tmp_pop[choices[idiff]]-tmp_pop[choices[idiff+1]]))
                    if self.scheme=='rand':
                        mutant[ind] = tmp_pop[choices[0]] + diffs
                    elif self.scheme=='best':
                        mutant[ind] = self.population[best_idx] + diffs
            # keep the bounds
            mutant = NN_DE.keep_bounds(self, mutant, bounds=[-1,1], idx=best_idx)

            # ============ Crossover Step ============= 
            trial_pop = np.copy(self.population)
            K = np.random.choice(trial_pop.shape[1])
            for ind in range(trial_pop.shape[0]):
                if gen_rand() < 0.1:
                    NCr[ind] = 0.8 +0.2*gen_rand()
                else:
                    NCr[ind] = self.Cr[ind]
                for jnd in range(trial_pop.shape[1]):
                    if jnd == K or gen_rand()<NCr[ind]:
                        trial_pop[ind][jnd] = mutant[ind][jnd]
            # keep the bounds
            trial_pop = NN_DE.keep_bounds(self, trial_pop, bounds=[-1,1], idx=best_idx)

            trial_fitness = NN_DE.set_weights_to_keras_model_and_compute_fitness(self, pop=trial_pop,
                                                                                 data=dataset,
                                                                                 nn_model=self.model)
            self.sp_evals += self.population.shape[0]
            
            # ============ Selection Step ==============
            winners = np.where(trial_fitness[:,0]>fitness[:,0])
        
            # Auto-adtaptation of F and Cr like NSSDE
            self.F[winners] = NF[winners]
            self.Cr[winners] = NCr[winners]
            
            # Greedy Selection
            fitness[winners] = trial_fitness[winners]
            self.population[winners] = trial_pop[winners]
            best_idx = np.argmax(fitness[:,0])
            
            if self.interactions > 0.95*self.max_sp_evals/self.n_pop:
                print('=====Interaction: {}====='.format(self.interactions+1))
                print('Best NN found - SP: {} / PD: {} / FA: {}'.format(fitness[best_idx][0],
                                                                        fitness[best_idx][1],
                                                                        fitness[best_idx][2]))
            
                #print('Test > Mean - SP: {} +- {}'.format(np.mean(test_fitness,axis=0)[0],
                #                                                    np.std(test_fitness,axis=0)[0]))
                
            # Local search like NSSDE 
            a_1 = gen_rand(); a_2 = gen_rand()
            a_3 = 1.0 - a_1 - a_2
            
            k, r1, r2 = np.random.choice(self.population.shape[0], size=3)
            V = np.zeros_like(self.population[k])
            for jdim in range(self.population.shape[1]):
                V[jdim] = a_1*self.population[k][jdim] + a_2*self.population[best_idx][jdim] + a_3*(self.population[r1][jdim] - self.population[r2][jdim])
                V = NN_DE.keep_bounds(self, V, bounds=self.bounds, idx=best_idx)
            
            
            V_train_fitness = NN_DE.set_weights_to_keras_model_and_compute_fitness(self, pop=V,
                                                                                   data=dataset,
                                                                                   nn_model=self.model)
            
            self.sp_evals += 1
            if V_train_fitness[0] > fitness[k][0]:
                #print('Found best model using local search...')
                self.population[k] = V
                if V_train_fitness[0] > fitness[best_idx][0]:
                    best_idx = k
            
            # ======== Done interaction ===========
            self.interactions += 1
            
            r_NNDE['log'].append((self.sp_evals, fitness[best_idx], np.mean(fitness, axis=0),
                             np.std(fitness, axis=0), np.min(fitness, axis=0), np.median(fitness, axis=0), self.F, self.Cr))
            
            #r_NNDE['test_log'].append((self.sp_evals, test_fitness[best_idx], np.mean(test_fitness, axis=0),
            #                 np.std(test_fitness, axis=0), np.min(test_fitness, axis=0), np.median(test_fitness, axis=0), self.F, self.Cr))
            
            #print('Fitness: ', fitness[:,0])
            #print('Mean: ',np.mean(fitness[:,0]))
            if np.mean(fitness[:,0]) > .9 and np.abs(np.mean(fitness[:,0])-fitness[best_idx][0])< self.sp_tol:
                print('Stop by Mean Criteria...')
                break
        # Compute the test
        #test_fitness = NN_DE.set_weights_to_keras_model_and_compute_fitness(self, pop=self.population,
        #                                                               data=self.test_dataset, nn_model=self.model)

        r_NNDE['champion weights'] = NN_DE.convert_vector_weights(self, self.population[best_idx], self.model)
        r_NNDE['model'] = self.model
        r_NNDE['best index'] = best_idx
        r_NNDE['Best NN'] = fitness[best_idx]
        r_NNDE['fitness'] = fitness
        #r_NNDE['test_fitness'] = test_fitness
        r_NNDE['population'] = self.population,
        return r_NNDE

Using Theano backend.
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/home/micael/anaconda3/envs/CodeLab/lib/python3.6/site-packages/theano/gpuarray/__init__.py", line 227, in <module>
    use(config.device)
  File "/home/micael/anaconda3/envs/CodeLab/lib/python3.6/site-packages/theano/gpuarray/__init__.py", line 214, in use
    init_dev(device, preallocate=preallocate)
  File "/home/micael/anaconda3/envs/CodeLab/lib/python3.6/site-packages/theano/gpuarray/__init__.py", line 99, in init_dev
    **args)
  File "pygpu/gpuarray.pyx", line 658, in pygpu.gpuarray.init
  File "pygpu/gpuarray.pyx", line 587, in pygpu.gpuarray.pygpu_init
pygpu.gpuarray.GpuArrayException: b'Could not load "libcuda.so": libnvidia-fatbinaryloader.so.384.130: cannot open shared object file: No such file or directory'


In [8]:
data = np.load('/home/micael/MyWorkspace/RingerRepresentation/2channels/data17-18_13TeV.sgn_lhmedium_probes.EGAM2.bkg.vetolhvloose.EGAM7.samples.npz')
sgn = data['signalPatterns_etBin_2_etaBin_0']
bkg = data['backgroundPatterns_etBin_2_etaBin_0']

# Equilibrate the classes to make a controled tests
bkg = bkg[np.random.choice(bkg.shape[0], size=sgn.shape[0]),:]
#print(sgn.shape, bkg.shape)

sgn_trgt = np.ones(sgn.shape[0])
bkg_trgt = -1*np.ones(bkg.shape[0])

sgn_normalized = np.zeros_like(sgn)
for ind in range(sgn.shape[0]):
    sgn_normalized[ind] = sgn[ind]/np.abs(np.sum(sgn[ind]))
    
bkg_normalized = np.zeros_like(bkg)
for ind in range(bkg.shape[0]):
    bkg_normalized[ind] = bkg[ind]/np.abs(np.sum(bkg[ind]))

data_ = np.append(sgn_normalized, bkg_normalized, axis=0)
trgt = np.append(sgn_trgt, bkg_trgt)

In [3]:
n_runs = 10

In [ ]:
%%time

result_dict = {}
nn_de = NN_DE(n_pop=20, max_sp_evals=2e3, scheme='best', sp_tol=1e-3)
for irun in range(n_runs):
    init_run_time = time.time()
    print('Begin Run {}'.format(irun+1))
    result_dict['Run {}'.format(irun+1)] = nn_de.evolution(dataset=(data_, trgt))
    end_run_time = time.time()
    print('Run {} - Time: {}'.format(irun+1, end_run_time - init_run_time))

In [12]:
return_dict.keys()

['Run 8',
 'Run 10',
 'Run 4',
 'Run 9',
 'Run 2',
 'Run 1',
 'Run 6',
 'Run 7',
 'Run 5',
 'Run 3']

In [13]:
r = []
m = []
for ifold in return_dict.keys():
    print(ifold, '>', return_dict[ifold]['fitness'][return_dict[ifold]['best index']])
    r.append(return_dict[ifold]['fitness'][return_dict[ifold]['best index']])
    m.append(np.mean(return_dict[ifold]['fitness'], axis=0))
    #print('population: {}+-{}'.format(np.around(np.mean(return_dict[ifold]['fitness'], axis=0),7),
    #      np.around(np.std(return_dict[ifold]['fitness'], axis=0),7)))
print(np.around(100*np.mean(r, axis=0),7), np.around(100*np.std(r, axis=0),7))
print('Pop: ', np.around(100*np.mean(m, axis=0),7), np.around(100*np.std(m, axis=0),7))

Run 8 > [ 0.90788686  0.92687464  0.11090441]
Run 10 > [ 0.90601017  0.93216943  0.11977676]
Run 4 > [ 0.9056507   0.9184316   0.10704064]
Run 9 > [ 0.91943109  0.92229536  0.08342873]
Run 2 > [ 0.91953231  0.93231254  0.0931597 ]
Run 1 > [ 0.91060131  0.92129365  0.10002862]
Run 6 > [ 0.92377269  0.93345736  0.08586148]
Run 7 > [ 0.9299779   0.95091586  0.09072696]
Run 5 > [ 0.93047869  0.94347453  0.08242702]
Run 3 > [ 0.93550415  0.95492272  0.08371494]
[ 91.8884586  93.3614768   9.5706926] [ 1.0434345  1.1875513  1.2476335]
Pop:  [ 91.867213   93.4919147   9.7405552] [ 1.0184136  1.0893271  1.0670021]


In [ ]:
r_ = {}
for ifold in return_dict.keys():
    print(len(return_dict[ifold]['log']))
    checks = list(range(0,len(return_dict[ifold]['log'])))
    r_[ifold]={}
    r_[ifold]['train'] = []
    #r_[ifold]['test'] = []
    for icheck in checks:
        #print(ifold, '>', return_dict[ifold]['log'][icheck][2])
        r_[ifold]['train'].append(return_dict[ifold]['log'][icheck][2])
        #r_[ifold]['test'].append(return_dict[ifold]['test_log'][icheck][2])

In [ ]:
merits = {
    'SP'  : 'SP Index',
    'PD'  : 'PD',
    'FA'  : 'FA'
}

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('_classic_test')

folds = ['Run 1', 'Run 2', 'Run 3', 'Run 4', 'Run 5', 'Run 6', 'Run 7', 'Run 8', 'Run 9', 'Run 10']
#x_axis = np.array([0., 20, 100, 1000,2000,3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000])

for idx, imerit in enumerate(merits.keys()):
    print('Plot: ', imerit)
    #f, (ax1, ax2) = plt.subplots(1, 2, sharey=False, figsize=(15,5))
    for ifold in folds:

        plt.plot(np.array(r_[ifold]['train'])[:,idx], label=ifold)
        plt.legend(fontsize='large', loc='best')
        plt.title(merits[imerit]+' - Train', fontsize=15)
        plt.xlabel('Interactions', fontsize=10)
        plt.ylabel('Mean '+merits[imerit], fontsize=10)
        plt.grid(True)

        #ax2.plot(np.array(r_[ifold]['test'])[:,idx], label=ifold)
        #ax2.legend(fontsize='large', loc='best')
        #ax2.set_title(merits[imerit]+' - Test', fontsize=15)
        #ax2.set_xlabel('Interactions', fontsize=10)
        #ax2.set_ylabel('Mean '+merits[imerit], fontsize=10)
        #ax2.grid(True)
    #plt.savefig(merits[imerit]+'.rand1bin.2000evals.withLS.MeanStopCriteria.pdf',)
    #plt.savefig(merits[imerit]+'.rand1bin.2000evals.withLS.MeanStopCriteria.png', dpi=150)
    plt.show()

In [ ]:
print(type(return_dict))
return_dict = dict(return_dict)
print(type(return_dict))

In [ ]:
return_dict.keys()

In [ ]:
return_dict['CVO'] = CVO

In [ ]:
return_dict['CVO']

In [ ]:
import pickle

with open('nnde.5neurons.rand1bin.2000evals.withLS.MeanStopCriteria.pickle', 'wb') as handle:
    pickle.dump(return_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Here begin the Backpropagation

Steps:

1. Get the champions and set the model.
2. Fit the model in each fold.
3. Get the results.

In [ ]:
number_of_epoch = 100

In [ ]:
%%time
for i in range(number_of_epoch):
    pass

In [ ]:
return_dict['Fold 1'].keys()

In [ ]:
return_dict['Fold 1']['champion weights']

In [ ]:
import time
inicio = time.time()
nn_de = NN_DE(n_pop=20, max_sp_evals=1e4, scheme='rand')
resultado = {}
for ifold, (train_index, test_index) in enumerate(CVO):
    print("TRAIN:", train_index, "TEST:", test_index, "Fold: ", ifold)
    resultado['Fold {}'.format(ifold+1)] = nn_de.evolution(train_dataset=(data_[train_index], trgt[train_index]),
                                                           test_dataset=(data_[test_index], trgt[test_index]))
fim=time.time()

print('Demorou - {} segundos'.format(fim-inicio))

In [ ]:
resultado['Fold 1']

In [ ]:
resultado = {}
for train_index, test_index in skf.split(data_, trgt):
    print("TRAIN:", train_index, "TEST:", test_index)
    

In [ ]:
teste = NN_DE(n_pop=20,max_sp_evals=2e3, scheme='rand', sp_tol=0.1)

In [ ]:
ev = teste.evolution(train_dataset=(data_, trgt), test_dataset=(data_, trgt))

In [ ]:
ev.keys()

In [ ]:
ev['log'][-1]

In [ ]:
np.mean(ev['fitness'],axis=0), np.std(ev['fitness'], axis=0)

In [ ]:
np.argmin(ev['fitness'][:,0]), ev['best index']

In [ ]:
ev['fitness'][ev['best index']], ev['fitness'][np.argmin(ev['fitness'][:,0])]